<a href="https://colab.research.google.com/github/abhijha8287/ann/blob/master/ann2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook guides you through a churn prediction project using the Bank Customer Churn dataset. It covers data loading, exploratory data analysis, preprocessing, building and evaluating an Artificial Neural Network (ANN) model, and comparing its performance to an XGBoost classifier.

# Download the dataset from kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauravtopre/bank-customer-churn-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/bank-customer-churn-dataset


# Import necessary lib

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhishekj15","key":"3342a20a434813170dc9f4c26ff63191"}'}

In [ ]:
import os

dataset_path = "/kaggle/input/bank-customer-churn-dataset"
os.listdir(dataset_path)


['Bank Customer Churn Prediction.csv']

# load the dataset

In [ ]:
import pandas as pd

dataset_path = "/kaggle/input/bank-customer-churn-dataset"
df = pd.read_csv(f"{dataset_path}/Bank Customer Churn Prediction.csv")
df.head()


,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.shape

(10000, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [ ]:
df.drop('customer_id', axis=1, inplace=True)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
credit_score,10000.0,650.528800,96.653299,350.00,584.00,652.000,718.0000,850.00
age,10000.0,38.921800,10.487806,18.00,32.00,37.000,44.0000,92.00
tenure,10000.0,5.012800,2.892174,0.00,3.00,5.000,7.0000,10.00
balance,10000.0,76485.889288,62397.405202,0.00,0.00,97198.540,127644.2400,250898.09
products_number,10000.0,1.530200,0.581654,1.00,1.00,1.000,2.0000,4.00
credit_card,10000.0,0.705500,0.455840,0.00,0.00,1.000,1.0000,1.00
active_member,10000.0,0.515100,0.499797,0.00,0.00,1.000,1.0000,1.00
estimated_salary,10000.0,100090.239881,57510.492818,11.58,51002.11,100193.915,149388.2475,199992.48
churn,10000.0,0.203700,0.402769,0.00,0.00,0.000,0.0000,1.00


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data preprocessing

In [ ]:
cols_to_scale=['credit_score', 'age', 'tenure', 'balance', 'estimated_salary']
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df[cols_to_scale]=scaler.fit_transform(df[cols_to_scale])

In [ ]:
df.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,0.538,France,Female,0.324324,0.2,0.000000,1,1,1,0.506735,1
1,0.516,Spain,Female,0.310811,0.1,0.334031,1,0,1,0.562709,0
2,0.304,France,Female,0.324324,0.8,0.636357,3,1,0,0.569654,1
3,0.698,France,Female,0.283784,0.1,0.000000,2,0,0,0.469120,0
4,1.000,Spain,Female,0.337838,0.2,0.500246,1,1,1,0.395400,0


In [ ]:
df=pd.get_dummies(data=df, columns=['gender','country'])

In [ ]:
df.head()

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,gender_Female,gender_Male,country_France,country_Germany,country_Spain
0,0.538,0.324324,0.2,0.000000,1,1,1,0.506735,1,True,False,True,False,False
1,0.516,0.310811,0.1,0.334031,1,0,1,0.562709,0,True,False,False,False,True
2,0.304,0.324324,0.8,0.636357,3,1,0,0.569654,1,True,False,True,False,False
3,0.698,0.283784,0.1,0.000000,2,0,0,0.469120,0,True,False,True,False,False
4,1.000,0.337838,0.2,0.500246,1,1,1,0.395400,0,True,False,False,False,True


# Spliting the data in train and test set

In [ ]:
x=df.drop('churn', axis=1)
y=df['churn']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape, x_test.shape,y_train.shape,y_test.shape

((8000, 13), (2000, 13), (8000,), (2000,))

# Set the hyperparameter of model and train the model

In [ ]:
import tensorflow as tf
from tensorflow import keras

model=keras.Sequential([
    keras.layers.Dense(13,input_shape=(13,),activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(9,activation='relu'),
    keras.layers.Dense(8,activation='relu'),
    keras.layers.Dense(7,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(x_train,y_train,epochs=60)

Epoch 1/60


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7800 - loss: 0.5644
Epoch 2/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7974 - loss: 0.4635
Epoch 3/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8035 - loss: 0.4287
Epoch 4/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8344 - loss: 0.3860
Epoch 5/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8428 - loss: 0.3750
Epoch 6/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8519 - loss: 0.3647
Epoch 7/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8521 - loss: 0.3560
Epoch 8/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8448 - loss: 0.3649
Epoch 9/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8536 - loss: 0.3580
Epoch 10/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8485 - loss: 0.3558
Epoch 11/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8525 - loss: 0.3582
Epoch 12/60
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

In [ ]:
y_pred=model.predict(x_train)
y_pred[:5]

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[0.01054561],
       [0.06014417],
       [0.06711002],
       [0.11585736],
       [0.65463084]], dtype=float32)

In [ ]:
y_pred = (y_pred > 0.5).astype(int)

In [ ]:
accuracy_score(y_train,y_pred)

0.86525

# Comparing xgboost and ann

In [ ]:
import xgboost as xgb

In [ ]:
model1=xgb.XGBClassifier()
model1.fit(x_train,y_train)
accuracy_score(y_train,model1.predict(x_train))

0.95875